<a href="https://colab.research.google.com/github/gkoorsen/DiffDock-autodocking/blob/main/Smina_Multi_Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install the necessary packages and libraries

In [ ]:
!pip install biopython
!pip install wget
!pip install requests

In [ ]:
!apt-get -qq install -y cmake
!apt-get -qq install -y swig
!apt-get -qq install -y libeigen3-dev



In [ ]:
!wget https://github.com/openbabel/openbabel/archive/openbabel-2-4-1.tar.gz
!tar xzvf openbabel-2-4-1.tar.gz


In [ ]:
!mkdir openbabel-openbabel-2-4-1/build
%cd openbabel-openbabel-2-4-1/build
!cmake ../ -DPYTHON_BINDINGS=ON -DRUN_SWIG=ON
!make
!make install
%cd /content


In [ ]:
import os
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/lib'


In [ ]:
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina && chmod +x smina


In [ ]:
from Bio.PDB import *
import subprocess
import re
import pandas as pd
import requests
import time
from tqdm import tqdm
import os
from google.colab import files
from numpy import mean
from numpy import amin, amax, array
import numpy as np


# Define functions

In [1]:
class NoHeteroSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.get_id()[0] == " " else 0

from Bio.PDB import Chain

from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB.Entity import Entity



def prepare_structure(name, file, ligand_res_name):

  command = f"touch {name}_ligand.pdb"
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


  command = f"grep '^HETATM.*FSN' {file} > {name}_ligand.pdb"
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


  command = f'obabel {name}_ligand.pdb -O {name}_ligand.pdbqt -h'
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


  command = f"touch {name}_apo.pdb"
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


  command = f"grep -v '^HETATM' {file} > {name}_apo.pdb"
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


  command = f'obabel {name}_apo.pdb -O {name}_apo_prepared.pdbqt -h'
  process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
  process.wait()


def prepare_ligand(name, smiles):

    if smiles is not None:

        command = f'obabel -:\"{smiles}\" -O {name}.sdf --gen2D'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()


        command = f'obabel {name}.sdf -O {name}_3d.sdf --gen3D'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()

        command = f'obabel {name}_3d.sdf -O {name}.pdbqt'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        process.wait()

        # Print the output
        print(process.stdout.read().decode())



def dock(name, ligand_name, smiles):

    prepare_ligand(ligand_name,smiles, exhaustiveness = 8)

    # Define the command as a string
    command = f"/content/smina -r {name}_apo_prepared.pdbqt -l {ligand_name}.pdbqt --autobox_ligand {name}_ligand.pdbqt --autobox_add 8 --exhaustiveness {exhaustiveness} -o {name}_result.pdbqt"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()

    command = f"grep '^REMARK minimizedAffinity' {name}_result.pdbqt | head -n 1 | awk '{{print $3}}'"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    top_score = subprocess.check_output(command, shell=True)
    top_score = top_score.decode("utf-8").strip()

    return top_score


def download_pdb_file(pdb_id: str) -> str:

    PDB_DIR ="/tmp/pdb/"
    os.makedirs(PDB_DIR, exist_ok=True)

    # url or pdb_id
    if pdb_id.startswith('http'):
        url = pdb_id
        filename = url.split('/')[-1]
    elif pdb_id.endswith(".pdb"):
        return pdb_id
    else:
        if pdb_id.startswith("AF"):
            url = f'https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb'
        else:
            url = f'http://files.rcsb.org/view/{pdb_id}.pdb'
        filename = f'{pdb_id}.pdb'

    cache_path = os.path.join(PDB_DIR, filename)
    if os.path.exists(cache_path):
        return cache_path

    pdb_req = requests.get(url)
    pdb_req.raise_for_status()
    open(cache_path, 'w').write(pdb_req.text)
    return cache_path


NameError: ignored

#Upload Excel template file

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)


Saving Docking_template.xlsx to Docking_template.xlsx


In [ ]:
PDBs = set([str(p).strip() for p in df['PDBs'].dropna()])
smiles = df['SMILES']
names = [f'Compound_{i}' for i in range(len(smiles))]
ligand_tags = df['Ligands']
compound2name = {c : n for c,n in zip(names,df['Compound names'])}

In [ ]:
PDB_files = [download_pdb_file(p) for p in tqdm(PDBs)]

100%|██████████| 1/1 [00:00<00:00, 1371.14it/s]


#Perform Docking

In [ ]:
top_affinities = {}

for (n,p,l) in zip(PDBs,PDB_files,ligand_tags):
    print(f'\t Preparing receptor {n}....')
    prepare_structure(n,p,l)
    for m,s,c in zip(names,smiles,df['Compound names']):
        if s != None:
            top_affinities[(n,m)] = dock(n,m,s)
            print(f'-{c}: {top_affinities[(n,m)]}')
